In [1]:
import numpy as np
import gym
import torch
import json
import numpy as np
from tqdm import trange

import maml_rl.envs
from maml_rl.baseline import LinearFeatureBaseline
from maml_rl.samplers import MultiTaskSampler
from maml_rl.utils.helpers import get_policy_for_env, get_input_size
from maml_rl.utils.reinforcement_learning import get_returns, reinforce_loss

from mandril.test import test
import multiprocessing as mp

In [2]:
with np.load("./maml-banditk5n10/results.npz", allow_pickle=True) as data_file:
    print(data_file.files)
    logs = {
        "tasks": data_file["tasks"],
        "train_returns" : data_file["train_returns"],
        "train_actions" : data_file["train_actions"],
        "valid_returns" : data_file["valid_returns"],
        "valid_actions" : data_file["valid_actions"],
    }

display(logs["tasks"].shape)
display(logs["train_returns"].shape)
display(logs["valid_returns"].shape)

display(logs["tasks"][0])
display(logs["train_returns"][0])
display(logs["valid_returns"][0])

# display(np.mean(logs["train_returns"],1))
# display(np.mean(logs["valid_returns"],1))

['tasks', 'train_returns', 'train_actions', 'valid_returns', 'valid_actions']


(10,)

(10, 10)

(10, 10)

{'mean': array([0.80739044, 0.51450007, 0.19111815, 0.18687478, 0.77252164])}

array([1., 0., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

array([2., 2., 2., 2., 2., 1., 1., 2., 2., 2.], dtype=float32)

In [11]:
with np.load("./maml-banditk5n10/results_maml_10000.npz", allow_pickle=True) as data_file:
    print(data_file.files)
    logs_maml = {
        "tasks": data_file["tasks"],
        "train_returns" : data_file["train_returns"],
        "train_actions" : data_file["train_actions"],
        "valid_returns" : data_file["valid_returns"],
        "valid_actions" : data_file["valid_actions"],
    }

display(logs_maml["tasks"].shape)
display(logs_maml["train_returns"].shape)
display(logs_maml["valid_returns"].shape)

display(logs_mandril["tasks"][0])
display(logs_mandril["train_returns"][0])
display(logs_mandril["train_actions"][0])
display(logs_mandril["valid_returns"][0])
display(logs_mandril["valid_actions"][0][0])
display(logs_mandril["valid_actions"][0][1])

['tasks', 'train_returns', 'train_actions', 'valid_returns', 'valid_actions']


(200,)

(200, 10)

(200, 10)

{'mean': array([0.80739044, 0.51450007, 0.19111815, 0.18687478, 0.77252164])}

array([1., 0., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

array([2., 2., 2., 2., 2., 1., 1., 2., 2., 2.], dtype=float32)

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.], dtype=float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [9]:
with np.load("./maml-banditk5n10/results_mandril_10000.npz", allow_pickle=True) as data_file:
    print(data_file.files)
    logs_mandril = {
        "tasks": data_file["tasks"],
        "train_returns" : data_file["train_returns"],
        "train_actions" : data_file["train_actions"],
        "valid_returns" : data_file["valid_returns"],
        "valid_actions" : data_file["valid_actions"],
    }

display(logs_mandril["tasks"].shape)
display(logs_mandril["train_returns"].shape)
display(logs_mandril["valid_returns"].shape)

display(logs_mandril["tasks"][0])
display(logs_mandril["train_returns"][0])
display(logs_mandril["train_actions"][0])
display(logs_mandril["valid_returns"][0])
display(logs_mandril["valid_actions"][0][1])


['tasks', 'train_returns', 'train_actions', 'valid_returns', 'valid_actions']


(200,)

(200, 10)

(200, 10)

{'mean': array([0.80739044, 0.51450007, 0.19111815, 0.18687478, 0.77252164])}

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [5]:
display(logs_mandril["tasks"])

array([{'mean': array([0.80739044, 0.51450007, 0.19111815, 0.18687478, 0.77252164])},
       {'mean': array([0.05182789, 0.06952621, 0.51517512, 0.68854365, 0.45962809])},
       {'mean': array([0.90201124, 0.77526743, 0.17072396, 0.0287679 , 0.88951437])},
       {'mean': array([0.41380904, 0.47149298, 0.50865694, 0.82380108, 0.82630446])},
       {'mean': array([0.58309365, 0.04657283, 0.80719875, 0.81909882, 0.06596405])},
       {'mean': array([0.62596875, 0.20261247, 0.75556707, 0.76133462, 0.67852638])},
       {'mean': array([0.98929989, 0.78344473, 0.17092267, 0.30795453, 0.59467745])},
       {'mean': array([0.83026257, 0.70099696, 0.17912511, 0.84094207, 0.646397  ])},
       {'mean': array([0.09988836, 0.38001795, 0.2485202 , 0.29715164, 0.53607668])},
       {'mean': array([0.05736228, 0.06603221, 0.36890483, 0.29402008, 0.60312307])},
       {'mean': array([0.19492033, 0.82676929, 0.80659698, 0.69124188, 0.23431798])},
       {'mean': array([0.01921456, 0.43279951, 0.10904